In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from deap import base, creator, tools
from sys import exit
from icecream import ic
import random

In [ ]:
def drawdown(max, min):
    return ((max - min) / max) *100

def drawdown_flag_init(idx):
    while True: #Descobre se o primeiro é maximo ou minimo, caso seja igual continua até encontrar um diferente
        if(df['Close'].size == idx+1):
            return False
        if(df['Close'].iloc[idx] > df['Close'].iloc[idx + 1]): #Caso em que o primeiro é máximo
            flag = False        #Quando False quer dizer que o próximo a encontrar é um minimo
            break
        
        elif(df['Close'].iloc[idx] < df['Close'].iloc[idx + 1]): #Caso em que o primeiro é minimo
            flag = True         #Quando True quer dizer que o próximo a encontrar é um máximo
            break

        idx+=1
    return flag

def pre_processing_drawdown(df):
    Array_max = []
    Array_min = []
    flag_max = drawdown_flag_init(0)
    if (flag_max == True):
        Array_min = np.append(Array_min, 0)
    else:
        Array_max = np.append(Array_max, 0)

    for idx, value in enumerate(df['Close']):
        if(df['Close'].size == idx+1):
            break               
        if(flag_max == True and value > df['Close'].iloc[idx+1]):
            Array_max = np.append(Array_max, idx) 
            flag_max = False      
        elif(flag_max == False and value < df['Close'].iloc[idx+1]):
            Array_min = np.append(Array_min, idx)
            flag_max = True
    return Array_max, Array_min

def find_max_drawdown(data, start, end):
    max_drawdown = 0
    peak = None
    for i in range(start, end):
        if peak is None or data.iloc[i] > data.iloc[peak]:
            peak = i
        elif (data.iloc[peak] - data.iloc[i])/data.iloc[peak] *100 > max_drawdown:
            max_drawdown = (data.iloc[peak] - data.iloc[i])/data.iloc[peak] *100
    return max_drawdown

def find_min_drawdown(data, start, end):
    max_drawdown = 0
    peak = None
    for i in range(start, end):
        if peak is None or data.iloc[i] < data.iloc[peak]:
            peak = i
        elif (data.iloc[i] - data.iloc[peak])/data.iloc[peak] *100 > max_drawdown:
            max_drawdown = (data.iloc[i]- data.iloc[peak])/data.iloc[peak] *100

    return max_drawdown

In [ ]:
start = pd.to_datetime('01-01-2023', dayfirst=True)
end = pd.to_datetime('01-06-2023', dayfirst=True) 
df = pd.read_csv('Data/AAPL.csv', sep=';', decimal='.',usecols=['Date', 'Close'])
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
df = df[(df['Date'] >= start) & (df['Date'] <= end)]
diff = pd.Series(df['Close'].diff())
df['Gain'] = diff.where(diff > 0, 0)
df['Loss'] = abs(diff.where(diff < 0, 0))
a = 40
b = 71
# df.to_csv("Teste2.csv")
print(df['Close'].iloc[a])
# plt.plot(df['Date'].iloc[a:b], df['Close'].iloc[a:b])
# plt.show()


In [ ]:
import bisect

sorted_list = [1, 3, 5, 7, 9, 11, 13]
# Find the index of the next number greater than or equal to a given value
value_to_find = 0
next_index = bisect.bisect_right(sorted_list, value_to_find)

print("Index of the next number greater than or equal to", value_to_find, "is:", next_index)

#garantir que se o index for maior então não aceder à list
sorted_list[next_index]

In [ ]:

def index_find(left, right, sorted_list, value_to_find):
    mid = (right - left)//2 + left
    if mid == right:
        return mid
    elif value_to_find < sorted_list[mid]:
        return index_find(left, mid, sorted_list, value_to_find)
    else:
        return index_find(mid+1, right, sorted_list, value_to_find)
    
sorted_list = [1, 3, 5, 6, 7, 9, 11]
for i in range(0,10):
    next_index = index_find(0, len(sorted_list), sorted_list, i)
    print(i , sorted_list[next_index])

In [ ]:
def pre_processing(df):
    Array_max = []
    Array_min = []
    idx = 0
    while True: #Encontrar se o primeiro é maximo ou minimo, caso igual continua até encontrar um diferente
        if(df['Close'].iloc[idx] > df['Close'].iloc[idx +1]):
            flag_max = False        #Quando false quer dizer que o próximo a encontrar é um minimo
            Array_max = np.append(Array_max, idx)
            break
        elif(df['Close'].iloc[idx] < df['Close'].iloc[idx + 1]):
            flag_max = True         #Quando True quer dizer que o próximo a encontrar é um máximo
            Array_min = np.append(Array_min, idx)
            break
        idx+=1

    for idx, value in enumerate(df['Close']):
        if(df['Close'].size == idx+1):
            break               
        if(flag_max == True and value > df['Close'].iloc[idx+1]):
            Array_max = np.append(Array_max, idx) 
            flag_max = False      
        elif(flag_max == False and value < df['Close'].iloc[idx+1]):
            Array_min = np.append(Array_min, idx)
            flag_max = True
    return Array_max, Array_min 

start = pd.to_datetime('01-01-2021', dayfirst=True)
end = pd.to_datetime('15-04-2021', dayfirst=True)
df = pd.read_csv('Data/AAPL.csv', sep=';', decimal='.',usecols=['Date', 'Close'])
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
df = df[(df['Date'] >= start) & (df['Date'] <= end)]
diff = pd.Series(df['Close'].diff())
df['Gain'] = diff.where(diff > 0, 0)
df['Loss'] = abs(diff.where(diff < 0, 0))
plt.plot(df['Date'], df['Close'])
Array_max, Array_min = pre_processing(df)
plt.scatter(df['Date'].iloc[Array_max.astype(int)], df['Close'].iloc[Array_max.astype(int)], color='red')
plt.scatter(df['Date'].iloc[Array_min.astype(int)], df['Close'].iloc[Array_min.astype(int)], color='green')
plt.show()

In [ ]:
#meter também o roi results a retornar   short, long 


# print("Para o path:", i)
#         max_short = 0
#         for k in range(0,3):
#             for l in range(0,21):
#                 for j in range(0,21):
#                     a= short_results(k,l,j)
#                     if(a>max_short):
#                         max_short = a
#                         Best_genes_short = [(k+1)*7,l*5,j*5]
#         print("Melhor para short:", Best_genes_short, max_short)
#         max_long = 0
# for k in range(0,3):
        #     for l in range(0,21):
        #         for j in range(0,21):
        #             a = long_results(k,l,j)
        #             if(a>max_long):
        #                 max_long = a
        #                 Best_genes_long = [(k+1)*7,l*5,j*5]    
        # print("Melhor para long:", Best_genes_long, max_long) 
        # print("Melhor ROI:", ROI_total(max_short, max_long))

In [1]:
# path = ['AAL', 'AAPL', 'AMZN', 'BAC', 'F', 'GOOG', 'IBM', 'INTC', 'NVDA', 'XOM']
path = ['IBM', 'INTC', 'NVDA', 'XOM']
for i in path:
    print("\n----------------------", i, "----------------------")
    Value_Roi = np.load("Resultados/ValuesNSGA2/Val_ROI_" + str(i) +".npy")
    Value_DD = np.load("Resultados/ValuesNSGA2/Val_DD_" + str(i) +".npy")
    MAX = np.max(Value_Roi[:,0])	
    mask = np.array(Value_Roi[:,0] == MAX)
    masked_array = Value_Roi[mask,1]
    MIN_value_DD = np.min(masked_array) #Menor valor para o maior ROI
            

    #Quando tem vários valores minimos iguais
    MIN  = np.min(Value_DD[:,1])
    mask2 = np.array(Value_DD[:,1] == MIN)
    masked_array2 = Value_DD[mask2,0]
    Max_value_ROI = np.max(masked_array2)  #Maior valor para o menor drawdown
    print("MAX -> ROI:", MAX, "DD:", MIN_value_DD)
    print("MIN -> ROI:", Max_value_ROI, "DD:", MIN)


---------------------- IBM ----------------------


NameError: name 'np' is not defined

In [ ]:
path = ['AAL', 'AAPL', 'AMZN', 'BAC', 'F', 'GOOG', 'IBM', 'INTC', 'NVDA', 'XOM']
# import locale
# locale.setlocale(locale.LC_ALL, 'fr_PT') 
for i in path:
    print("\n----------------------", i, "----------------------")
    Value_train = np.load('Resultados/Value/Val_train_'+ i +'.npy')
    MAX = np.max(Value_train)
    MIN  = np.min(Value_train)
    STD = np.std(Value_train)
    MEAN = np.mean(Value_train)
    # MAX = locale.format("%f", MAX, grouping=True)
    # MIN = locale.format("%f", MIN, grouping=True)
    # STD = locale.format("%f", STD, grouping=True)
    # MEAN = locale.format("%f", MEAN, grouping=True)
    print("\nValores para o periodo de treino do Path ", i, ":")
    print("MAX:", MAX)
    print("MIN:", MIN)
    print("Mean:", MEAN)
    print("STD:", STD)

In [ ]:
path = ['AAL', 'AAPL', 'AMZN', 'BAC', 'F', 'GOOG', 'IBM', 'INTC', 'NVDA', 'XOM']

for i in path:
    print("\n----------------------", i, "----------------------")
    Value_test = np.load('Resultados/Value/Val_test_'+ i +'.npy')
    MAX2 = np.max(Value_test)
    MIN2  = np.min(Value_test)
    STD2 = np.std(Value_test)
    MEAN2 = np.mean(Value_test)
    print("\nValores para o periodo de teste do Path ", i, ":")
    print("MAX:", MAX2)
    print("MIN:", MIN2)
    print("Mean:", MEAN2)
    print("STD:", STD2)